In [40]:
"""
    Convert excel to markdown
"""
import os
from pathlib import Path
from docling.document_converter import DocumentConverter

file_path = '../data/Banking Statement.xlsx'

def get_docling_result(path):
    if not os.path.exists(path):
        print(f"Error: File does not exist {path}")
        return None

    try:
        converter = DocumentConverter()
        markdown_text = converter.convert(path)
        return markdown_text
    except Exception as e:
        print(f"Error while converting: {e}")
        return None

markdown_data = get_docling_result(file_path)
if markdown_data:
    result = markdown_data.document.export_to_markdown()
    print(f"Convert to markdown successfully")
    # print(result)
else:
    print(f"Parse markdown was getting error")


Convert to markdown successfully


In [52]:
import re

markdown_text = result

lines = markdown_text.splitlines()

# Find the header row index
header_idx = None
for i, line in enumerate(lines):
    if "Ngày giao dịch" in line and "Số tham chiếu" in line:
        header_idx = i
        break

if header_idx is None:
    raise ValueError("❌ Could not find transaction table header")

# Parse header
header_line = lines[header_idx]
headers = [h.strip() for h in header_line.strip("|").split("|")]

print("Headers:")
print(headers)

# Parse data rows
sample_dataset = []
for line in lines[header_idx + 2:]:
    if "Tổng số" in line:
        break
    if not line.strip().startswith("|"):
        continue

    cells = [c.strip() for c in line.strip("|").split("|")]

    # Skip empty / invalid rows
    if len(cells) != len(headers):
        continue

    row_dict = dict(zip(headers, cells))
    sample_dataset.append(row_dict)

print("Total transaction rows:", len(sample_dataset))
print("Example of sample_dataset:")
print(sample_dataset[1])


Headers:
['Ngày giao dịch', 'Số tham chiếu', 'Số tiền ghi nợ', 'Số tiền ghi có', 'Mô tả']
Total transaction rows: 41
Example of sample_dataset:
{'Ngày giao dịch': '2024-04-30 00:00:00', 'Số tham chiếu': '9925 - 34742', 'Số tiền ghi nợ': 'None', 'Số tiền ghi có': '6563913', 'Mô tả': 'T/t T/ung the VISA:SIEU THI BIG C AN LAC BA RIA; MerchNo:11500000457 Gross Amt: On-Us=6,630,215.00 VND; VAT Amt:6,630,215.00*1.000%/11 = 6,027.00 VND(VAT code:0301472278); Code:1005; SLGD:On-Us=7; Ngay 30/04/2024.'}


In [53]:
def extract_banking_info(markdown_text):
    def find_value(label):
        m = re.search(rf"{re.escape(label)}:\s*\|\s*([^|\n]+)", markdown_text)
        return m.group(1).strip() if m else ""

    common_info = {
        "Chủ tài khoản": find_value("Chủ tài khoản"),
        "Số tài khoản": find_value("Số tài khoản"),
        "Địa chỉ": find_value("Địa chỉ"),
        "CIF": find_value("CIF"),
        "Loại tiền": find_value("Loại tiền"),
    }

    return common_info

banking_info = extract_banking_info(markdown_text)
print(banking_info)


{'Chủ tài khoản': 'CT TNHH TM& DV SIEU THI BIG C AN LAC', 'Số tài khoản': '71004403751', 'Địa chỉ': '1231 QLO 1A,KP5,F.BINHTRI DONG B,BINHTAN', 'CIF': '101346', 'Loại tiền': 'VND'}


In [54]:
import json

tasks = []
task_id = 1

for row in sample_dataset:
    date_raw = row.get("Ngày giao dịch", "")
    if date_raw:
        # convert 2024-04-30 00:00:00 -> 30/04/2024
        try:
            parts = date_raw.split(" ")[0].split("-")
            date_str = f"{parts[2]}/{parts[1]}/{parts[0]}"
        except:
            date_str = date_raw
    else:
        date_str = ""

    text_block = (
        f"Ngày giao dịch: {date_str}\n"
        f"Số tham chiếu: {row.get('Số tham chiếu','')}\n"
        f"Số tiền ghi có: {row.get('Số tiền ghi có','')}\n"
        f"Chủ tài khoản: {banking_info['Chủ tài khoản']}\n"
        f"Số tài khoản: {banking_info['Số tài khoản']}\n"
        f"CIF: {banking_info['CIF']}\n"
        f"Địa chỉ: {banking_info['Địa chỉ']}\n"
        f"Loại tiền: {banking_info['Loại tiền']}\n"
        f"Mô tả: {row.get('Mô tả','')}"
    )

    task = {
        "id": task_id,
        "data": {
            "text": text_block
        }
    }

    tasks.append(task)
    task_id += 1

print("Total Label sample_dataset:", len(tasks))
print("Example of sample_dataset:\n")
print(tasks[0]["data"]["text"])


Total Label sample_dataset: 41
Example of sample_dataset:

Ngày giao dịch: 30/04/2024
Số tham chiếu: 9925 - 34743
Số tiền ghi có: 9798337
Chủ tài khoản: CT TNHH TM& DV SIEU THI BIG C AN LAC
Số tài khoản: 71004403751
CIF: 101346
Địa chỉ: 1231 QLO 1A,KP5,F.BINHTRI DONG B,BINHTAN
Loại tiền: VND
Mô tả: T/t T/ung the VISA:SIEU THI BIG C AN LAC BA RIA; MerchNo:11500000457 Gross Amt: Not On-Us=9,907,318.00 VND; VAT Amt:9,907,318.00*1.100%/11 = 9,907.00 VND(VAT code:0301472278); Code:1005; SLGD: Not On-Us=13; Ngay 30/04/2024.


In [55]:
output_path = "../dataset/sample/base_data_label.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(tasks, f, ensure_ascii=False, indent=2)

print("Saved to:", output_path)


Saved to: ../dataset/sample/base_data_label.json


In [56]:
import random

header_mapping = {
    "Ngày giao dịch": ["Ngày giao dịch", "Ngày GD", "Date", "Trans Date", "Ngày"],
    "Số tham chiếu": ["Số tham chiếu", "Mã giao dịch", "Ref No", "Reference", "Số lệnh"],
    "Số tiền ghi có": ["Số tiền ghi có", "Số tiền nhận", "Credit", "Amount", "Số tiền"],
    "VAT Amt": ["VAT Amt", "Thuế VAT", "VAT Amount"],
    "Gross Amt": ["Gross Amt", "Tổng số tiền", "Gross Amount"],
    "Chủ tài khoản": ["Chủ tài khoản", "Tên tài khoản", "Account Name", "Beneficiary"],
    "Số tài khoản": ["Số tài khoản", "Số TK", "Account Number", "STK"],
    "CIF": ["CIF", "Mã khách hàng", "Mã CIF"],
    "Địa chỉ": ["Địa chỉ", "Address", "Location"],
    "Loại tiền": ["Loại tiền", "Currency", "Đơn vị"],
    "Mô tả": ["Mô tả", "Nội dung", "Description", "Nội dung thanh toán"]
}


In [57]:
import json

augmented_tasks = []
task_id = 1
AUG_PER_ROW = 5

for row in sample_dataset:
    # Prepare base values
    date_raw = row.get("Ngày giao dịch", "")
    if date_raw:
        try:
            parts = date_raw.split(" ")[0].split("-")
            date_val = f"{parts[2]}/{parts[1]}/{parts[0]}"
        except:
            date_val = date_raw
    else:
        date_val = ""

    base_values = {
        "Ngày giao dịch": date_val,
        "Số tham chiếu": row.get("Số tham chiếu", ""),
        "Số tiền ghi có": row.get("Số tiền ghi có", ""),
        "VAT Amt": "",
        "Gross Amt": "",
        "Chủ tài khoản": common_meta["Chủ tài khoản"],
        "Số tài khoản": common_meta["Số tài khoản"],
        "CIF": common_meta["CIF"],
        "Địa chỉ": common_meta["Địa chỉ"],
        "Loại tiền": common_meta["Loại tiền"],
        "Mô tả": row.get("Mô tả", "")
    }

    # Extract VAT and Gross from description
    desc = base_values["Mô tả"]
    if desc:
        vat_match = re.search(r"VAT Amt[:=]\s*([\d,\.]+)", desc)
        gross_match = re.search(r"Gross Amt[:=]\s*([\d,\.]+)", desc)

        if vat_match:
            base_values["VAT Amt"] = vat_match.group(1)
        if gross_match:
            base_values["Gross Amt"] = gross_match.group(1)

    # Generate data for sample_dataset
    for _ in range(AUG_PER_ROW):
        lines = []

        for canonical_header, value in base_values.items():
            if not value:
                continue

            # Pick a random header
            header_variants = header_mapping[canonical_header]
            random_header = random.choice(header_variants)

            lines.append(f"{random_header}: {value}")

        # Shuffle line order as robustness
        random.shuffle(lines)

        text_block = "\n".join(lines)

        task = {
            "id": task_id,
            "data": {
                "text": text_block
            }
        }

        augmented_tasks.append(task)
        task_id += 1

print("Total sample_dataset has augmented:", len(augmented_tasks))
print("\nExample of sample_dataset was augmented:\n")
print(augmented_tasks[0]["data"]["text"])


Total sample_dataset has augmented: 205

Example of sample_dataset was augmented:

Account Number: 71004403751
Description: T/t T/ung the VISA:SIEU THI BIG C AN LAC BA RIA; MerchNo:11500000457 Gross Amt: Not On-Us=9,907,318.00 VND; VAT Amt:9,907,318.00*1.100%/11 = 9,907.00 VND(VAT code:0301472278); Code:1005; SLGD: Not On-Us=13; Ngay 30/04/2024.
Date: 30/04/2024
Address: 1231 QLO 1A,KP5,F.BINHTRI DONG B,BINHTAN
Loại tiền: VND
Mã giao dịch: 9925 - 34743
Mã khách hàng: 101346
Account Name: CT TNHH TM& DV SIEU THI BIG C AN LAC
Amount: 9798337
VAT Amt: 9,907,318.00


In [58]:
output_path = "../dataset/label/ready_dataset_label.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(augmented_tasks, f, ensure_ascii=False, indent=2)

print("Saved augmented dataset to:", output_path)


Saved augmented dataset to: ../dataset/label/ready_dataset_label.json
